# Microsoft-Vision-ResNet50 based Hybrid Model using Linear Classifier

In [1]:
#conda install pytorch torchvision cudatoolkit=10.1 -c pytorch

#pip install progressbar2


In [2]:

# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier/'
assert FOLDERNAME is not None, "[!] Enter the foldername."


import sys
sys.path.append('/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier')

%cd /$FOLDERNAME/datasets/
!bash get_datasets.sh
%cd /$FOLDERNAME

/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier/datasets
/home/ubuntu/Vision-Classifiers/Microsoft-Vision-Classifier


In [3]:

import numpy as np
import matplotlib.pyplot as plt
from data_utils import get_CIFAR10_data

%load_ext autoreload
%autoreload 2

def calculate_error(x, y):
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [4]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

import progressbar
from progressbar import progressbar

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression

In [5]:
import microsoftvision

In [6]:
class Wrangling:
    def __init__(self):
        self.wrangling = transforms.Compose([
                                           transforms.Resize(224),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])])

    def __call__(self, x):
        return self.wrangling(x)[[2,1,0],:,:]

In [7]:

data = get_CIFAR10_data()
for k, v in list(data.items()):
    print(f"{k}: {v.shape}")

X_train: (49000, 3, 32, 32)
y_train: (49000,)
X_val: (1000, 3, 32, 32)
y_val: (1000,)
X_test: (1000, 3, 32, 32)
y_test: (1000,)


In [8]:


train = CIFAR10('./path', download=True, train=True, transform=Wrangling())
test = CIFAR10('./path', download=True, train=False, transform=Wrangling())

Files already downloaded and verified
Files already downloaded and verified


In [9]:
model = microsoftvision.models.resnet50(pretrained=True)

Loading Microsoft Vision pretrained model
Model already downloaded.


In [10]:
model.eval()
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [11]:
def extractor(dataset, model):
    all_features = []
    all_labels = []
    import time
    import logging
    import progressbar
    torch.cuda.empty_cache()

    with torch.no_grad():
        for point, labels in progressbar.progressbar(DataLoader(dataset, batch_size=128, num_workers=8)):
            point = point.cuda()
            labels = labels.cuda()
            features = model(point)

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [12]:
train_features, train_labels = extractor(train, model)
test_features, test_labels = extractor(test, model)

100% (391 of 391) |######################| Elapsed Time: 0:04:57 Time:  0:04:57
100% (79 of 79) |########################| Elapsed Time: 0:00:59 Time:  0:00:59


In [13]:
classifier = KNeighborsClassifier(n_neighbors=5, n_jobs=16)


In [14]:
classifier.fit(train_features, train_labels)
predictions = classifier.predict(test_features)
result = 100 * np.mean((test_labels == predictions).astype(np.float))
print(f"Hybrid Model Performance Accuracy: {result}")

Hybrid Model Performance Accuracy: 92.92
